# Algoritmo RSA

El algotirmo RSA es un algoritmo utilizado en encriptacion de mensajes de clave publica, desarrollado en 1979 por Ronald L. Rivest, Adi Shamir y Leonard Adleman. La seguridad de dicho algoritmo se basa en la factorizacion de numeros enteros principalmente numeros primos grandes muy grandes, actualmente se manejan numeros hasta de 300 digitos, por este motivo RSA brinda una buena seguridad ya que es bastante dificil descomponer estos numeros primos aleatorios a fuerza bruta, hacerlo tiene un costo de tiempo computacional demasiado alto.

El siguiente codigo es la funcion de hallar el maximo comun divisor, dicha funcion nos servira para identificar si dos numeros son cooprimos, dos numeros son cooprimos si su mcd = 1.

In [ ]:
import random
from sympy import mod_inverse

In [ ]:
#funcion para hallar el maximo comun divisor
def mcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

La siguiente funcion es para evaluar si un numero es primo utilizando la criba de Eratóstenes, solo que en este caso no guardaremos toda la lista de numeros primos antes del numero que se pasa en el argumento de esta funcion, si no que solo evaluaremos este numero con la operacion modulo % y si su residuo es cero este numero no sera primo.

In [ ]:
def es_Primo(num):
    if num == 2:
        return True
    #aqui indicamos si el numero es menor que dos o el numero es par
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num ** 0.5) + 2, 2):
        if num % n == 0:
            return False
    return True

Ya teniendo las funciones de mcd y es_Primo procedemos a explicar el funcionamiento del algoritmo RSA, este algoritmo como es de llave publica cada usuario tiene dos llaves una publica y otra privada, basicamente el funcionamiento de este algoritmo trata de que al momento de enviar un mensaje, el emisor busca la llave publica del receptor y cifra dicho mensaje con esa llave, al momento que el receptor reciba dicho mensaje encriptado este lo desencriptara con su llave privada. Para realizar esto se usan conceptos de aritmetica modular y la funcion phi de euler los cuales se explicaran a continuación:

# 1. Generacion de llaves

Para la generacion de estas llaves se escogen dos numeros primos p y q aleatorios, en la practica estos numeros tienden a ser muy grandes hasta de 300 digitos, cabe aclarar que para la muestra de este algoritmo estos numeros no se generan si no que el usuario tiene que ingresarlos, esto lo hago para ahorrar tiempo de ejecucion del programa, luego de ingresar los numeros p y q y verificar que ambos son primos con la funcion es_Primo, se procede a hallar n el cual sera el modulo a manejar para la llave privada y la llave publica, donde n = p*q, luego hallamos el valor de la funcion \\(\phi\\) de euler, \\(\phi(n)\\) calcula el numero de enteros positivos menores o iguales a n y que sean cooprimos con n, esta funcion nos servira para hallar la llave privada d, mas adelante se mostrara como se logra esto, luego se utiliza dos propiedades que tiene dicha funcion las cuales son las siguientes:




$$
\text{1. } \phi(p) =\text{p-1 donde p es primo}\\ 
\text{2. si m y n son primos entre si, entonces } \phi(mn) = \phi(m)\phi(n)
$$

Por lo tanto \\(\phi\\) de euler  es igual a \\(\phi(n) = (p-1)*(q-1)\\), luego procedemos a hallar la llave publica e la cual tiene que ser un numero entero positivo menor que \\(\phi(n)\\) y que sea cooprimo a este, si esta llave e es un numero pequeño el cifrado sera mas rapido pero mas inseguro, pero si esta e es mas grande el cifrado sera mas lento pero mas seguro.

Luego procedemos a hallar la llave privada d, mediante aritmetica modular, donde e y d satisfagan la siguiente congruencia:  e*d \\(\equiv 1 (mod \phi(n))\\), en pocas palabras que d sea el multiplicador inverso de \\(e (mod \phi(n))\\), para hallar este valor se utiliza el algoritmo de Euclides extendido.

Entoces resumiendo las llaves se guardan de la siguiente forma (e,n) donde e = llave publica (exponente de cifrado) y n es el modulo y (d,n) donde d = llave privada (exponente de descifrado) y n es el modulo, todos estos valores se guardan secretamente.

Como nota cabe aclarar que el algoritmo ha mejorado en rendimiento y se utilizan otros metodos mas eficientes, como por ejemplo ya no se usa la funcion \\(\phi\\) de euler si no la funcion de Carmichael, esta funcion no la explicare ya que el codigo implementado no hace uso de esta funcion, solo lo comento para tener en cuenta en dado caso que alguien quiera hacerle una mejora al codigo.

In [ ]:
def generadorDeLlaves(p, q):
    if not (es_Primo(p) and es_Primo(q)):
        raise ValueError('Ambos numeros deben ser primos.')
    elif p == q:
        raise ValueError('p y q no deben ser iguales')
    # n = pq
    n = p * q

    # Phi es la funcion totiente  de euler de n
    phi = (p - 1) * (q - 1)

    # Escoge un entero e tal que e y phi(n) sean cooprimos
    e = random.randrange(1, phi)
    print('e= ',e)


    # usamos el algoritmo de Euclides para verificar que e y phi(n) son cooprimos
    g = mcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = mcd(e, phi)

    # Usamos el algoritmo extendido de euclides para generar la llave privada
    #usamos la libreria de sympy ya que tiene implementada la funcion de hallar el inverso modulo n
    d= mod_inverse(e,phi)

    # Retorna un par de llaves
    # llave publica es : (e, n) y la llave privada es: (d, n)
    return ((e, n), (d, n))

# 2. Cifrado

El emisor envia un mensaje y este se cifra usando la siguiente operacion de aritmetica modular \\(c\equiv m^{e} (mod (n))\\) donde m es el mensaje y donde c va ser el mensaje ya cifrado, si nos fijamos aqui se utiliza la llave publica e y el modulo n para cifrar.

In [ ]:
def encriptar(pk, textoPlano):

    key, n = pk
    #convierte cada letra in del texto plano en numeros basados de los caracteres de la tabla ASCII usando a^b mod m
    cifrado = [(ord(char) ** key) % n for char in textoPlano]
    #retorna un arreglo de bytes
    return cifrado

# 3. Descifrado

Ahora el receptor puede descifrar el mensaje enviado por el emisor, haciendo uso de su llave privada, mediante la siguiente operacion \\(m\equiv c^{d} (mod (n))\\)

Lo anterior funciona debido a que \\(c^{d}= (m^{e})^{d}\equiv m^{ed} (mod (n))\\), y esto sucede debido a que seleccionamos a d y e de forma que ed = 1 +k\\(\phi(n)\\), aplicando el teorema de euler se tiene lo siguiente: 

\\(m^{ed} = m^{1+k\phi(n)} = m(m^{\phi(n)})^{k}\equiv m (mod(n)) \\).
Ya con esto obtenemos el mensaje descifrado.

In [ ]:
def desencriptar(pk, textoCifrado):
    key, n = pk
    #Genera el texto plano basado en el texto cifradoo y la llave usando a^b mod m
    plain = [chr((char ** key) % n) for char in textoCifrado]
    #Retorna un arreglo de bytes como un string
    return ''.join(plain)

In [ ]:
p = int(input("Ingrese un numero primo (17, 19, 23, etc): "))
q = int(input("Ingrese otro numero primo que no sea igual primer numero que ingreso:  "))
print ("Espere un momento estamos generando su par de llaves publica y privada . . .")
publica, privada = generadorDeLlaves(p, q)
print ("Su llave publica es:  ", publica ," y su llave privada es:  ", privada)
mensaje = input("Ingrese un mensaje para encriptar con su llave publica: ")
mensajeEncriptado = encriptar(publica, mensaje)
print ("Su mensaje encriptado es: ")
print (''.join(map(lambda x: str(x), mensajeEncriptado)))
print ("Desencriptando mensaje con la llave privada ", privada ," . . .")
print ("Su mensaje es: ")
print (desencriptar(privada, mensajeEncriptado))